In [24]:
from pyspark.sql import SparkSession
sparkSession = SparkSession.builder.enableHiveSupport().master("local").getOrCreate()
from pyspark.sql.functions import row_number, col, rank, when, sum, abs, count, lit, round

alpha = 0.15

In [51]:
from pyspark.sql.types import *
schema = StructType([
StructField("id1", IntegerType(), True),
StructField("id2", IntegerType(), True),
StructField("beta_weight", DoubleType(), True),])

l = [(1, 2, 0.4), (1, 3, 0.24), (1, 5, 0.36), (2, 3, 0.7), (2, 4, 0.3), (3, 5, 1.0), (4, 2, 0.3), (4, 5, 0.7), (5, 3 ,1.0)]
edges1 = sparkSession.createDataFrame(l, schema)

l1 = [(1, 1),(2,0),(3,0),(4,0),(5,0)]
schema1 = StructType([
StructField("id", IntegerType(), True),
StructField("p_u", IntegerType(), True)])
u = sparkSession.createDataFrame(l1, schema1)

l2 = [(1, 1),(2,1),(3,1),(4,0),(5,1)]
schema2 = StructType([
StructField("id", IntegerType(), True),
StructField("p", IntegerType(), True)])
x_0 = sparkSession.createDataFrame(l2, schema2)

In [26]:
x_0.join(edges1, col("id") == col("id1"), "left")\
    .withColumn("next_vertex", col("p") * col("beta_weight")).collect()

[Row(id=1, p=1, id1=1, id2=2, beta_weight=0.4, next_vertex=0.4),
 Row(id=1, p=1, id1=1, id2=3, beta_weight=0.24, next_vertex=0.24),
 Row(id=1, p=1, id1=1, id2=5, beta_weight=0.36, next_vertex=0.36),
 Row(id=3, p=1, id1=3, id2=5, beta_weight=1.0, next_vertex=1.0),
 Row(id=5, p=1, id1=5, id2=3, beta_weight=1.0, next_vertex=1.0),
 Row(id=4, p=0, id1=4, id2=2, beta_weight=0.3, next_vertex=0.0),
 Row(id=4, p=0, id1=4, id2=5, beta_weight=0.7, next_vertex=0.0),
 Row(id=2, p=1, id1=2, id2=3, beta_weight=0.7, next_vertex=0.7),
 Row(id=2, p=1, id1=2, id2=4, beta_weight=0.3, next_vertex=0.3)]

In [27]:
x_0.join(edges1, col("id") == col("id1"), "left")\
    .withColumn("next_vertex", col("p") * col("beta_weight"))\
    .groupBy("id2").agg(sum("next_vertex")).collect()

[Row(id2=3, sum(next_vertex)=1.94),
 Row(id2=5, sum(next_vertex)=1.3599999999999999),
 Row(id2=4, sum(next_vertex)=0.3),
 Row(id2=2, sum(next_vertex)=0.4)]

In [28]:
sigma1 = x_0.join(edges1, col("id") == col("id1"), "left")\
    .withColumn("next_vertex", col("p") * col("beta_weight"))\
    .groupBy("id2").agg(sum("next_vertex").alias("sigma"))

In [29]:
x_1 = u.join(sigma1, col("id")==col("id2"), "left").na.fill(0, "sigma")\
        .select("id", col("p_u")*alpha + (1-alpha)*col("sigma")).orderBy("id")
x_1.collect()

[Row(id=1, ((p_u * 0.15) + (sigma * 0.85))=0.15),
 Row(id=2, ((p_u * 0.15) + (sigma * 0.85))=0.34),
 Row(id=3, ((p_u * 0.15) + (sigma * 0.85))=1.649),
 Row(id=4, ((p_u * 0.15) + (sigma * 0.85))=0.255),
 Row(id=5, ((p_u * 0.15) + (sigma * 0.85))=1.156)]

In [ ]:
for i in range(5):
    sigma1 = x_0.join(edges1, col("id") == col("id1"), "left")\
    .withColumn("next_vertex", col("p") * col("beta_weight"))\
    .groupBy("id2").agg(sum("next_vertex").alias("sigma"))
    
    x_0 = u.join(sigma1, col("id")==col("id2"), "left").na.fill(0, "sigma")\
       .select("id", (col("p_u")*alpha + (1-alpha)*col("sigma")).alias("p")).orderBy("id")
    x_0.collect()

In [ ]:
sigma1.collect()

[Row(id2=3, sigma=1.3806508923125),
 Row(id2=5, sigma=1.10221169975),
 Row(id2=4, sigma=0.0175633576875),
 Row(id2=2, sigma=0.06559280025)]